In [1]:
!pip install lxml
!pip install geocoder
!pip install folium
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import geocoder 
import folium # map rendering library
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

     |████████████████████████████████| 102kB 8.5MB/s ta 0:00:011
     |████████████████████████████████| 92kB 5.8MB/s eta 0:00:011


Read the table on the website and assign it to dataframe df

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = tables[0]
df.head(20)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


Drop all rows with value == "Not assigned" of column Borough

In [3]:
df = df[df["Borough"]!= "Not assigned"]
df.head(20)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


Replace all Neighbohoods with value "Not assigned" with it Borough


In [4]:
df.loc[df["Neighborhood"]=="Not assigned","Neighborhood"]= df.loc[df["Neighborhood"]=="Not assigned","Borough"]
df.head(20)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


Save all Postcode which are multiple in the table to a dictionary. The postcode is the key and the Borough is the value

In [5]:
postCodes = df["Postcode"].values.T.tolist()

postCodesNTimes = {}
for i in range(1,len(postCodes)):
    postCode = postCodes[i]
    for j in  range(i+1,len(postCodes)):
            if postCode == postCodes[j]:
                postCodesNTimes[postCode]=df.iloc[j,1]
    
postCodesNTimes

{'M6A': 'North York',
 'M1B': 'Scarborough',
 'M4B': 'East York',
 'M5B': 'Downtown Toronto',
 'M9B': 'Etobicoke',
 'M1C': 'Scarborough',
 'M3C': 'North York',
 'M9C': 'Etobicoke',
 'M1E': 'Scarborough',
 'M3H': 'North York',
 'M5H': 'Downtown Toronto',
 'M6H': 'West Toronto',
 'M2J': 'North York',
 'M3J': 'North York',
 'M5J': 'Downtown Toronto',
 'M6J': 'West Toronto',
 'M1K': 'Scarborough',
 'M3K': 'North York',
 'M4K': 'East Toronto',
 'M5K': 'Downtown Toronto',
 'M6K': 'West Toronto',
 'M1L': 'Scarborough',
 'M2L': 'North York',
 'M4L': 'East Toronto',
 'M5L': 'Downtown Toronto',
 'M6L': 'North York',
 'M1M': 'Scarborough',
 'M2M': 'North York',
 'M5M': 'North York',
 'M6M': 'York',
 'M9M': 'North York',
 'M1N': 'Scarborough',
 'M6N': 'York',
 'M1P': 'Scarborough',
 'M5P': 'Central Toronto',
 'M6P': 'West Toronto',
 'M1R': 'Scarborough',
 'M5R': 'Central Toronto',
 'M6R': 'West Toronto',
 'M9R': 'Etobicoke',
 'M5S': 'Downtown Toronto',
 'M6S': 'West Toronto',
 'M1T': 'Scarborough'